## **Imports and configurations**

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import time

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from IPython.display import IFrame
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

from plot_utils import *

In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import shuffle
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

In [4]:
folder_path = 'C:/Users/yaass/OneDrive/Bureau/Parser'

## **Models evaluation**

In [5]:
def preview_data(folder_path, file_name, index_value = 'md5'):
    df = pd.read_csv(os.path.join(folder_path, file_name), index_col=index_value)
    print(f'Nb Observations: {df.shape[0]}')
    print(f'Nb Columns: {df.shape[1]}')
    return df.head(3)


def get_X_y(folder_path, file_name, index_value = 'md5'):
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path, index_col = index_value)
    X = df.drop('label', axis=1)
    y = df['label']
    return shuffle(X, y)


def get_models():
    models = dict()
    models['Logistic Regression'] = LogisticRegression()
    models['K-Nearest Neighbors'] = KNeighborsClassifier()
    models['Decision tree'] = DecisionTreeClassifier()
    models['Random Forest'] = RandomForestClassifier()
    models['Stochastic Gradient Descent'] = SGDClassifier()
    models['SVM'] = SVC()
    models['Gaussian Naive Bayes'] = GaussianNB()
    return models


def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores


def get_evaluation_results(folder_path, file_name):  
    
    #generate features and target
    X, y = get_X_y(folder_path, file_name)
    
    #generate models
    models = get_models()
    
    #evaluate models and store results
    results, names = list(), list()
    for name, model in models.items():
        scores = evaluate_model(model, X, y)
        results.append(scores)
        names.append(name) 
        
    return names, results


def print_evaluation_results(results, names):  
    for name, scores in zip(names, results):
        print(f"{name:30} \t: {np.mean(scores)*100:.3f}% ( {np.std(scores)*100:.3f}% )")  

### 1. apistats (one-hot encoded)

In [30]:
file_name = 'onehot_encoded_apistats_dataset.csv'

preview_data(folder_path, file_name)

Nb Observations: 3761
Nb Columns: 304


,LdrUnloadDll,RegCloseKey,GetSystemTimeAsFileTime,LoadStringW,GetSystemInfo,RegQueryValueExA,LdrGetProcedureAddress,MessageBoxTimeoutW,RegSetValueExA,NtTerminateProcess,...,CIFrameElement_CreateElement,CDocument_write,WSAConnect,CopyFileExW,NtDeleteFile,ExitWindowsEx,CreateServiceW,WNetGetProviderNameW,RtlCreateUserProcess,NtLoadKeyEx
md5,,,,,,,,,,,,,,,,,,,,,
00263ca2071dc9a6ee577eb356b0d1d9,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
0137996cd3aa197ae8eb64fef12c044a,0,0,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
0282f83bbfb58c08b54dbd8015e54d2e,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
start = time.time()

names, results = get_evaluation_results(folder_path, file_name)

print(f'Cross-validation {len(names)} models required a duration of {time.time()-start:.2f} seconds')

Cross-validation 7 models required a duration of 47.15 seconds


In [32]:
print_evaluation_results(results, names)

Logistic Regression            	: 91.084% ( 1.210% )
K-Nearest Neighbors            	: 91.686% ( 1.199% )
Decision tree                  	: 90.977% ( 1.418% )
Random Forest                  	: 94.788% ( 1.021% )
Stochastic Gradient Descent    	: 91.137% ( 1.652% )
SVM                            	: 93.043% ( 1.278% )
Gaussian Naive Bayes           	: 69.698% ( 2.623% )


In [33]:
fig = plot_evaluation_boxplots(results, names, title = 'Models Performance On One-Hot Encoded "apistats" Data', y_axis = 'Accuracy')

figure_path = 'figures/evaluation_onehot_apistats_data.html'

save_figures_to_html(figure_path, [fig])

IFrame(figure_path, width=900, height=600)

### 1. apistats (count encoded)

In [6]:
file_name = 'count_encoded_apistats_dataset.csv'

preview_data(folder_path, file_name)

Nb Observations: 3822
Nb Columns: 305


,LdrUnloadDll,RegCloseKey,GetSystemTimeAsFileTime,LoadStringW,GetSystemInfo,RegQueryValueExA,LdrGetProcedureAddress,MessageBoxTimeoutW,RegSetValueExA,NtTerminateProcess,...,CDocument_write,WSAConnect,ExitWindowsEx,CopyFileExW,NtDeleteFile,CreateServiceW,WNetGetProviderNameW,RtlCreateUserProcess,NtLoadKeyEx,NtLoadDriver
md5,,,,,,,,,,,,,,,,,,,,,
00263ca2071dc9a6ee577eb356b0d1d9,4,3,1,2,1,13,6,1,6,3,...,0,0,0,0,0,0,0,0,0,0
0137996cd3aa197ae8eb64fef12c044a,0,0,6,2,1,0,55,1,0,0,...,0,0,0,0,0,0,0,0,0,0
0282f83bbfb58c08b54dbd8015e54d2e,0,9,1,0,0,0,21,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
start = time.time()

names, results = get_evaluation_results(folder_path, file_name)

print(f'Cross-validation on {len(names)} models required a duration of {time.time()-start:.2f} seconds')

Cross-validation on 7 models required a duration of 105.51 seconds


In [8]:
print_evaluation_results(results, names)

Logistic Regression            	: 71.996% ( 2.046% )
K-Nearest Neighbors            	: 86.325% ( 1.383% )
Decision tree                  	: 90.808% ( 1.486% )
Random Forest                  	: 94.924% ( 0.858% )
Stochastic Gradient Descent    	: 68.629% ( 2.506% )
SVM                            	: 59.603% ( 0.635% )
Gaussian Naive Bayes           	: 56.689% ( 1.475% )


In [9]:
fig = plot_evaluation_boxplots(results, names, title = 'Models Performance On Count Encoded "apistats" Data', y_axis = 'Accuracy')

figure_path = 'figures/evaluation_count_apistats_data.html'

save_figures_to_html(figure_path, [fig])

IFrame(figure_path, width=900, height=600)

### 3. dll_loaded (one-hot encoded)

In [19]:
file_name = 'onehot_encoded_dll_dataset.csv'

preview_data(folder_path, file_name)

Nb Observations: 3162
Nb Columns: 2242


,RTUTILS.DLL,kernel32.dll,label,kernel32,C:\Users\cucko\AppData\Local\Temp\emedcfd.dll,RpcRtRemote.dll,CRYPTSP.dll,CLBCatQ.DLL,msimg32.dll,libssl32.dll,...,C:\Users\cucko\AppData\Local\Temp\VmX.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_1d4457e8e6917937845f55ebbce2fc49.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_1d50b69a05d60d4f9f703b789a2933de.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_268eef019bf65b2987e945afaf29643f.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_3c1b2fabb7d74bc5be0820eae4107f8a.exe,C:\Users\cucko\AppData\Local\Temp\VirusShare_43b844c35e1a933e9214588be81ce772.dll,C:\Users\cucko\AppData\Local\Temp\VirusShare_933b11bc4799f8d9f65466fb2e3ea659.exe,C:\Users\cucko\AppData\Local\Temp\VirusShare_9fc3ed6c9b8056fbf155f79569ca7cb1.exe,D3DIM700.DLL,C:\Users\cucko\AppData\Local\Temp\VirusShare_bcbdef1678049378be04719ed29078d2.dll
md5,,,,,,,,,,,,,,,,,,,,,
00263ca2071dc9a6ee577eb356b0d1d9,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0137996cd3aa197ae8eb64fef12c044a,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0282f83bbfb58c08b54dbd8015e54d2e,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
start = time.time()

names, results = get_evaluation_results(folder_path, file_name)

print(f'Cross-validation {len(names)} models required a duration of {time.time()-start:.2f} seconds')

Cross-validation 7 models required a duration of 248.08 seconds


In [29]:
print_evaluation_results(results, names)

Logistic Regression            	: 87.150% ( 1.716% )
K-Nearest Neighbors            	: 84.282% ( 2.141% )
Decision tree                  	: 84.999% ( 1.983% )
Random Forest                  	: 87.307% ( 1.634% )
Stochastic Gradient Descent    	: 86.264% ( 1.738% )
SVM                            	: 87.624% ( 1.809% )
Gaussian Naive Bayes           	: 64.811% ( 2.367% )


In [23]:
fig = plot_evaluation_boxplots(results, names, title = 'Models Performance On Raw "dll_loaded" Data', y_axis = 'Accuracy')

figure_path = 'figures/evaluation_raw_dll_data.html'

save_figures_to_html(figure_path, [fig])

IFrame(figure_path, width=900, height=600)